In [1]:
!pip install transformers[sentencepiece]
!pip install datasets

     |████████████████████████████████| 4.0 MB 12.8 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 6.6 MB 37.9 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 12.2 MB/s 
     |████████████████████████████████| 136 kB 51.2 MB/s 
     |████████████████████████████████| 212 kB 52.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.1 MB/s 
     |████████████████████████████████| 127 kB 51.9 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 271 kB 47.0 MB/s 
  Attempting uninstall: urllib3
    Found e

In [6]:
import pandas as pd

from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset

from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_path = '/data/train.csv'
df = pd.read_csv(train_path)
df = df.fillna("NONE")

# Here I opt for a 85-15 split betwen train and development sets
train_df, dev_df = train_test_split(df,test_size=0.15,random_state=42)

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)

train_data = train_data.rename_columns({'TEXT':'text','LABEL':'label'})
dev_data = dev_data.rename_columns({'TEXT':'text','LABEL':'label'})

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

train_data = train_data.map(preprocess_function, batched=True)
dev_data = dev_data.map(preprocess_function, batched=True)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10

  0%|          | 0/60 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [9]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("/submission_checkpoint", local_files_only=True)
trainer = Trainer(model=model, data_collator = data_collator)
trainer.model = model.cuda()
y = trainer.predict(dev_data)

loading configuration file /content/drive/MyDrive/Ling539/submission_checkpoint/config.json
Model config DistilBertConfig {
  "_name_or_path": "/content/drive/MyDrive/Ling539/submission_checkpoint",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "vocab_size": 30522
}

loading weights file /content/drive/MyDrive/Ling539/submission_checkpoint/pytor

In [10]:
from sklearn.metrics import classification_report
import numpy as np

preds = np.argmax(y.predictions,axis=1)

def metric_printout(modelname:str, labels, preds):

    print(modelname,": -------------------")

    print(classification_report(labels,preds,target_names=['Not Movie','Positive','Negative']))

metric_printout("Transformer Evaluation", preds, dev_data['label'])

Transformer Evaluation : -------------------
              precision    recall  f1-score   support

   Not Movie       1.00      0.99      0.99      5267
    Positive       0.93      0.91      0.92      2768
    Negative       0.90      0.93      0.92      2494

    accuracy                           0.95     10529
   macro avg       0.94      0.94      0.94     10529
weighted avg       0.95      0.95      0.95     10529

